# GPU enabled Kubeflow notebook
In this notebook we demonstrate multi GPU training using tensorflow framework. In the example we use convolutional neural network for image classification trained on cifar10 dataset. 

Example was tested using a [Kubeflow Notebook](https://www.kubeflow.org/docs/components/notebooks/) in [Charmed Kubeflow](https://charmed-kubeflow.io/).  The notebook used here was deployed with 8 NVIDIA GPUs.

**Important**

This example was tested on drivers:
```
NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7
```
And ```tensorflow-gpu==2.10.0```. Check rest of the requirements in the attatched `requirements.txt`.

In [1]:
# Import libs and setup environment
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

# Set for optimized memory usage
# os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
# os.environ["NCCL_DEBUG"] = "WARN"

# Allow to use whole GPU memory
# config = tf.compat.v1.ConfigProto()
# config.gpu_options.allow_growth = True

tf.config.experimental.list_physical_devices()

2023-02-24 15:51:07.798852: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-24 15:51:07.953939: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-24 15:51:08.614887: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-24 15:51:08.615058: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load d

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]

In [2]:
# Extract GPU device names
device_type = 'GPU'
devices = tf.config.experimental.list_physical_devices(
          device_type)
devices_names = [d.name.split("e:")[1] for d in devices]
devices_names

['GPU:0', 'GPU:1', 'GPU:2', 'GPU:3', 'GPU:4', 'GPU:5', 'GPU:6', 'GPU:7']

In [3]:
# Image recognition dataset for 10 class prediction
# https://www.cs.toronto.edu/~kriz/cifar.html
# Image has 32x32x3 resolution
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
X_train.shape, X_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [4]:
# Scaling image values between 0-1
X_train_scaled = X_train/255
X_test_scaled = X_test/255

In [5]:
# One hot encoding labels
y_train_encoded = keras.utils.to_categorical(y_train, num_classes = 10, dtype = 'float32')
y_test_encoded = keras.utils.to_categorical(y_test, num_classes = 10, dtype = 'float32')

In [6]:
def get_model():
    '''
    Create simple deep model. 3 hidden layers. 
    '''
    strategy = tf.distribute.MirroredStrategy(
           devices=devices_names)
    with strategy.scope():
        model = keras.Sequential([
            keras.layers.Flatten(input_shape=(32,32,3)),
            keras.layers.Dense(3000, activation='relu'),
            keras.layers.Dense(1000, activation='relu'),
            keras.layers.Dense(10, activation='sigmoid')    
        ])
        model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [7]:
def get_model_cnn():
    '''
    Create More sophisticated model. Using mirrored distributed training strategy
    Reading: https://www.tensorflow.org/guide/distributed_training#mirroredstrategy
    '''
    strategy = tf.distribute.MirroredStrategy(
           devices=devices_names)
    with strategy.scope():
        model = keras.models.Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
        model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(10, activation='softmax'))
        # compile model
        opt = keras.optimizers.SGD(lr=0.001, momentum=0.9)
        model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
        return model

In [8]:
m = get_model_cnn()

2023-02-24 15:51:18.842049: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-24 15:51:22.965724: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38406 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2023-02-24 15:51:22.968850: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38406 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:0f:00.0, compute capability: 8.0
2023-02-24 15:51:22.971765: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/devi

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


/opt/conda/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [10]:
# MultiGPU prediction. Run nvidia-smi command in another terminal tab to check GPU ussage
batch_size_per_gpu = 32
n_gpus = len(devices_names)
model_gpu = get_model_cnn()
model_gpu.fit(X_train_scaled, y_train_encoded, epochs=100, batch_size=batch_size_per_gpu * n_gpus)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Epo

2023-02-24 15:51:44.273393: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-02-24 15:51:45.146582: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-02-24 15:51:46.105967: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-02-24 15:51:47.399241: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-02-24 15:51:48.558755: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-02-24 15:51:48.973089: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-02-24 15:51:49.983410: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-02-24 15:51:50.990104: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-02-24 15:51:52.059061: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN ve

196/196 [==============================] - 25s 11ms/step - loss: 2.0000 - accuracy: 0.2838
Epoch 2/100
196/196 [==============================] - 2s 9ms/step - loss: 1.7467 - accuracy: 0.3904
Epoch 3/100
196/196 [==============================] - 1s 7ms/step - loss: 1.6133 - accuracy: 0.4343
Epoch 4/100
196/196 [==============================] - 2s 9ms/step - loss: 1.5242 - accuracy: 0.4639
Epoch 5/100
196/196 [==============================] - 2s 8ms/step - loss: 1.4557 - accuracy: 0.4864
Epoch 6/100
196/196 [==============================] - 1s 7ms/step - loss: 1.4026 - accuracy: 0.5065
Epoch 7/100
196/196 [==============================] - 1s 7ms/step - loss: 1.3613 - accuracy: 0.5196
Epoch 8/100
196/196 [==============================] - 1s 7ms/step - loss: 1.3138 - accuracy: 0.5385
Epoch 9/100
196/196 [==============================] - 1s 8ms/step - loss: 1.2762 - accuracy: 0.5506
Epoch 10/100
196/196 [==============================] - 2s 8ms/step - loss: 1.2404 - accuracy: 0.5642

In [12]:
# # CPU only prediction
# %%timeit -n1 -r1
# # CPU
# with tf.device('/CPU:0'):
#     model_cpu = get_model()
#     model_cpu.fit(X_train_scaled, y_train_encoded, epochs = 1)

In [13]:
# # Single GPU example
# %%timeit -n1 -r1
# # GPU
# with tf.device('GPU:0'):
#     model_gpu = get_model()
#     model_gpu.fit(X_train_scaled[:500], y_train_encoded[:500], epochs = 10)

In [14]:
model_gpu.predict(np.concatenate([X_test_scaled, X_test_scaled, X_test_scaled, X_test_scaled, X_test_scaled])) # We multiplied the X_test to check GPU usage

1563/1563 [==============================] - 8s 4ms/step


array([[1.8274143e-08, 4.9641289e-07, 2.3056466e-06, ..., 3.1217842e-06,
        1.6067820e-06, 3.5467662e-07],
       [4.7272952e-05, 4.4401992e-02, 6.8558115e-12, ..., 1.6050975e-16,
        9.5554942e-01, 1.2385259e-06],
       [9.6274596e-01, 5.7169900e-04, 1.4223373e-06, ..., 2.5707018e-07,
        3.6452767e-02, 1.6477342e-04],
       ...,
       [1.6607229e-08, 7.4171147e-10, 2.4342506e-01, ..., 3.2161029e-06,
        1.6595010e-04, 1.0971085e-05],
       [3.8951438e-02, 2.3104828e-03, 8.3454677e-07, ..., 3.0839238e-09,
        1.4882334e-09, 5.3805156e-09],
       [6.7962800e-16, 1.1599146e-10, 3.5637537e-08, ..., 9.9998641e-01,
        7.0848181e-11, 4.4483093e-09]], dtype=float32)